# One Float Sampling

This notebook tests Metropolis-Hastings on the one-float problem.

In [1]:
using AutomotiveDrivingModels
using AutoScenes
using Base.Test

In [2]:
const FloatScene = Frame{Entity{Float64, Void, Int}};
function AutoScenes.Vars(scene::FloatScene, roadway::Void)
    x = scene[1].state
    return Vars(Float64[x], # vector containing the one variable
                [StateBounds(-x, 1-x)], # bounds are deltas so we can vary x by this much to remain in [0,1]
                [:x], # variable symbol
                [1])  # index of entity in the scene
end
function f1(
    vars::Vars,
    assignment::Assignment, # indeces of variables in vars
    roadway::Void,
    )::Float64

    return vars.values[assignment[1]] < 0.5
end
function f2(
    vars::Vars,
    assignment::Assignment, # indeces of variables in vars
    roadway::Void,
    )::Float64

    return vars.values[assignment[1]] ≥ 0.5
end;
function AutoScenes.assign_feature{F <: typeof(f1), Void}(
    f::F,
    scene::FloatScene,
    roadway::Void,
    vars::Vars,
    )

    return Assignment[(1,)]
end
function AutoScenes.assign_feature{F <: typeof(f2), Void}(
    f::F,
    scene::FloatScene,
    roadway::Void,
    vars::Vars,
    )

    return Assignment[(1,)]
end;

roadway = nothing
scenes = [
    FloatScene([Entity{Float64,Void,Int}(0.0,roadway,1)], 1),
    FloatScene([Entity{Float64,Void,Int}(1/4,roadway,1)], 1),
    FloatScene([Entity{Float64,Void,Int}(3/4,roadway,1)], 1),
    FloatScene([Entity{Float64,Void,Int}(3/4,roadway,1)], 1),
    FloatScene([Entity{Float64,Void,Int}(3/4,roadway,1)], 1),
    FloatScene([Entity{Float64,Void,Int}(1.0,roadway,1)], 1),
]
features = (f1, f2)
factorgraphs = [FactorGraph(features, scene, roadway) for scene in scenes];
model = FactorModel(features, [1.0,1.0]);

In [3]:
gen = FactorGraphSceneGenerator(model, Dict(:x => Normal(0.0,1.0)), 1000)

FactorGraphSceneGenerator(burnin=1000)

In [4]:
factorgraph = factorgraphs[1]
a = zeros(length(factorgraph.vars))
b = similar(a)
logPtilde_a = begin
    factorgraph.vars.values .+= a
    logPtilde_a = log_ptilde(gen.model.features, gen.model.weights, factorgraph.vars,
        factorgraph.assignments, factorgraph.roadway)
    factorgraph.vars.values .-= a
    return logPtilde_a
end

srand(0)
(a, logPtilde_a) = metropolis_hastings_step!(gen, factorgraph, a, b, logPtilde_a)

([0.0], 1.0)

In [5]:
metropolis_hastings!(gen, factorgraph)

1-element Array{Float64,1}:
 0.998656

In [10]:
samples = Array{Float64}(1000)
for i in 1 : length(samples)
    samples[i] = metropolis_hastings!(gen, factorgraph)[1]
end
using PGFPlots
Axis(Plots.Histogram(samples, discretization=:specified), ymin=0)

StackOverflowError: [91mStackOverflowError:[39m

In [7]:
Plo = count(samples .< 0.5)/length(samples)
@test isapprox(Plo, 0.5, atol=0.01)

Test Passed


In [12]:
model = FactorModel(features, [1.0,2.0])
gen = FactorGraphSceneGenerator(model, Dict(:x => Normal(0.0,1.0)), 1000)
samples = Array{Float64}(1000)
for i in 1 : length(samples)
    samples[i] = metropolis_hastings!(gen, factorgraph)[1]
end
using PGFPlots
Axis(Plots.Histogram(samples, discretization=:specified), ymin=0)

1000-element Array{Float64,1}:
 0.128752
 0.65427 
 0.558814
 0.94582 
 0.802543
 0.274318
 0.855546
 0.274562
 0.891642
 0.936697
 0.68031 
 0.297351
 0.634778
 ⋮       
 0.598059
 0.587871
 0.760793
 0.748314
 0.543621
 0.616776
 0.475922
 0.57919 
 0.224675
 0.601805
 0.0531  
 0.623853

In [9]:
P_target = exp(1) / (exp(2) + exp(1))
Plo = count(samples .< 0.5)/length(samples)
@test isapprox(P_target, P_target, atol=0.01)

Test Passed
